In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

from acquire import get_telco_data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customer_id                 7043 non-null   object 
 1   gender                      7043 non-null   object 
 2   senior_citizen              7043 non-null   int64  
 3   partner                     7043 non-null   object 
 4   dependents                  7043 non-null   object 
 5   tenure                      7043 non-null   int64  
 6   phone_service               7043 non-null   object 
 7   multiple_lines              7043 non-null   object 
 8   internet_service_type_id    7043 non-null   int64  
 9   online_security             7043 non-null   object 
 10  online_backup               7043 non-null   object 
 11  device_protection           7043 non-null   object 
 12  tech_support                7043 non-null   object 
 13  streaming_tv                7043 

In [3]:
telco=get_telco_data()
telco.head()

,Unnamed: 0,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id.1,contract_type,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type
0,0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,...,2,59.9,542.4,No,1,Month-to-month,1,DSL,2,Mailed check
1,1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,...,4,69.4,571.45,No,1,Month-to-month,1,DSL,4,Credit card (automatic)
2,2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,...,1,48.2,340.35,No,1,Month-to-month,1,DSL,1,Electronic check
3,3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,...,1,25.1,25.1,Yes,1,Month-to-month,1,DSL,1,Electronic check
4,4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,...,3,30.5,30.5,Yes,1,Month-to-month,1,DSL,3,Bank transfer (automatic)


In [17]:
telco= telco.drop(columns='Unnamed: 0')
telco= telco.drop(columns='internet_service_type_id.1')
telco= telco.drop(columns='payment_type_id')
telco= telco.drop(columns='contract_type_id.1')
telco= telco.drop(columns='payment_type_id.1')
telco= telco.drop(columns= 'contract_type_id')

In [18]:
telco.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,...,No,No,Yes,No,59.9,542.4,No,Month-to-month,DSL,Mailed check
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,...,Yes,Yes,Yes,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,...,No,No,No,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,...,No,No,No,Yes,25.1,25.1,Yes,Month-to-month,DSL,Electronic check
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,...,No,No,No,No,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)


In [21]:
telco_dummies = pd.get_dummies(telco.churn, drop_first=True)
telco_dummies.head(3)

,Yes
0,0
1,0
2,0


In [24]:
telco = pd.concat([telco, telco_dummies], axis=1)
telco.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,Yes,Yes,Yes
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,...,No,59.9,542.4,No,Month-to-month,DSL,Mailed check,0,0,0
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,No,No,...,Yes,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic),0,0,0
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,Yes,No,...,Yes,48.2,340.35,No,Month-to-month,DSL,Electronic check,0,0,0
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,No,No,...,Yes,25.1,25.1,Yes,Month-to-month,DSL,Electronic check,1,1,1
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,Yes,No,...,No,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic),1,1,1


In [35]:
def prep_telco(cached=True):
    '''
    This function acquires and prepares the iris data from a local csv, default.
    Passing cached=False acquires fresh data from Codeup db and writes to csv.
    Returns the iris df with dummy variables encoding species.
    '''
    
    # use my aquire function to read data into a df from a csv file
    df = get_telco_data()
    
    # drop and rename columns
    df= df.drop(columns='Unnamed: 0')
    df= df.drop(columns='internet_service_type_id.1')
    df= df.drop(columns='payment_type_id')
    df= df.drop(columns='contract_type_id.1')
    df= df.drop(columns='payment_type_id.1')
    df= df.drop(columns= 'contract_type_id')

    
    # create dummy columns for churn
    telco_dummies = pd.get_dummies(df.churn, drop_first=True)
    
    # add dummy columns to df
    df = pd.concat([df, telco_dummies], axis=1)
    df= df.rename(columns={'Yes': 'is_churn'})
    
    return df

In [36]:
prep_telco()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,is_churn
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,No,Yes,No,59.90,542.4,No,Month-to-month,DSL,Mailed check,0
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,Yes,Yes,Yes,69.40,571.45,No,Month-to-month,DSL,Credit card (automatic),0
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,No,No,Yes,48.20,340.35,No,Month-to-month,DSL,Electronic check,0
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,...,No,No,Yes,25.10,25.1,Yes,Month-to-month,DSL,Electronic check,1
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,Yes,...,No,No,No,30.50,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9950-MTGYX,Male,0,Yes,Yes,28,Yes,No,3,No internet service,...,No internet service,No internet service,Yes,20.30,487.95,No,Two year,None,Credit card (automatic),0
7039,9953-ZMKSM,Male,0,No,No,63,Yes,Yes,3,No internet service,...,No internet service,No internet service,No,25.25,1559.3,No,Two year,None,Mailed check,0
7040,9964-WBQDJ,Female,0,Yes,No,71,Yes,Yes,3,No internet service,...,No internet service,No internet service,Yes,24.40,1725.4,No,Two year,None,Credit card (automatic),0
7041,9972-EWRJS,Female,0,Yes,Yes,67,Yes,No,3,No internet service,...,No internet service,No internet service,Yes,19.25,1372.9,No,Two year,None,Bank transfer (automatic),0


In [ ]:
! git add "rep"